<a href="https://colab.research.google.com/github/HosakaKeigo/gmail-fetcher/blob/main/colab/faqMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備
- Vertex AIを有効化したGoogle Cloud Project
- Colabを実行するアカウントへのVertex AI Userを付与

In [ ]:
# config
GOOGLE_CLOUD_PROJECT_ID = "YOUR GOOGLE CLOUD PROJECT ID"
GEMINI_MODEL = "gemini-2.5-flash-preview-05-20"

# Apps Scriptを紐付けたシート
SHEET_ID = "YOUR SPEREAD_SHEET_ID"
FAQ_SHEET_NAME = "FAQ"

# カラム指定
THREAD_ID_COLUMN = 1
MAIL_COLUMN = 3

# 処理したFAQの保存シート
FAQ_SHEET_NAME_PROCESSED = "processed"

# 処理したThreadのIDを保存するシート
INDEX_SHEET = "indexed"

In [ ]:
%pip install genai

In [ ]:
SYSTEM_PROMPT = """あなたはFAQマニュアル作成アシスタントです。お問い合わせをもとに、一般化したFAQマニュアルを作成と、返信文面のマスキングを行なってください。

## FAQマニュアル作成手順
1. メールのやり取りを確認してください。
2. メールの内容から以下を抽出してください。
  - 質問内容（question）
  - 対応内容(todo)
    - 返信するまでに確認するべき事項・対応すべき事項を抽出してください。
    - なるべくmarkdownのリスト形式で、手順がわかりやすいようにしてください。
3. 返信文面のマスキングを行なってください。

## FAQマニュアル作成のポイント
- FAQマニュアルは、お問い合わせ内容を一般化したものです。個人名などの個人情報・個別情報は決して含めないでください。
- マニュアルに相応しい明確で簡潔な文体を心がけてください。
- 返信文面例は、実際の返信文面を抽象化して作成してください。

## 返信文面のマスキング
返信文面に対して以下を行なってください。

- 宛名などの個人名は「◆◆◆」で置き換えてください。
- 事例によって回答が異なる箇所は、<項目名>というプレースホルダを用いてください。
- その他については、オリジナルの文面を極力保ってください。

## 回答の形式
以下のJSON形式で回答を提出してください。

\`\`\`json
{
  "question": "質問内容。対応が変わる可能性がある条件は明記すること。",
  "todo": "対応内容",
  "reply": "返信例文。実際の返信文から個人情報をマスキングしたもの。"
}
\`\`\`

それでは、お問い合わせ文面を与えます。"""

In [ ]:
import re

def mask_email(email_body: str) -> str:
    """
    文字列内のメールアドレスを「◆◆◆」に置換してマスクする関数。

    Args:
        email_body (str): 入力文字列（メール本文など）

    Returns:
        str: メールアドレスがマスクされた文字列
    """
    pattern = r'\b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\b'
    return re.sub(pattern, "◆◆◆", email_body)

In [ ]:
import os
from google import genai
from google.colab import userdata, auth
from pydantic import BaseModel
import json

class FAQ(BaseModel):
    question: str
    todo: str
    reply: str

# Require Vertex AI User Role
auth.authenticate_user(
    project_id=GOOGLE_CLOUD_PROJECT_ID
)

def extract_faq(contents: str) -> str:
    client = genai.Client(
        vertexai=True,
        project=GOOGLE_CLOUD_PROJECT_ID,
        location='us-central1'
    )

    try:
        response = client.models.generate_content(
          model=GEMINI_MODEL,
          contents=mask_email(contents),
          config=genai.types.GenerateContentConfig(
              system_instruction=SYSTEM_PROMPT,
              temperature=0,
              response_mime_type='application/json',
              response_schema=FAQ,
          ),
        )
        return json.loads(response.text)
    except Exception as e:
        print(e)
        return

In [ ]:
# Sheet
from typing import List
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
ss = gc.open_by_key(SHEET_ID)
faq_sheet = ss.worksheet(FAQ_SHEET_NAME)
processed_sheet = ss.worksheet(FAQ_SHEET_NAME_PROCESSED)
indexed_sheet = ss.worksheet(INDEX_SHEET)

def get_faqs():
    try:
        values = faq_sheet.get_all_values()

        # Skip header row
        if len(values) > 0:
            return values[1:]
        return []

    except Exception as e:
        print(f"Error retrieving FAQs: {str(e)}")
        return []


def append_faq(rows: List[FAQ]):
    processed_sheet.append_rows(rows)

def append_indexed(filenames:List[List[str]]):
    indexed_sheet.append_rows(filenames)

def get_indexed_files() -> List[str]:
    return indexed_sheet.col_values(1)

In [ ]:
from typing import List
import json
from pydantic import BaseModel
from time import sleep

BATCH_SIZE = 5

def process_mail_faqs():
    batch_rows = []
    batch_indexed_files = []

    faq_rows = get_faqs()

    # すでに処理済みのThreadはスキップ
    indexed_files = get_indexed_files()

    for row in faq_rows:
        thread_id = row[THREAD_ID_COLUMN]
        if thread_id in indexed_files:
            print(f"Skipping row with ID {thread_id} as it has already been processed.")
            continue
        mail_content = row[MAIL_COLUMN]
        if not mail_content:
            print(f"Skipping row with ID {thread_id} as mail content is empty.")
            continue

        try:
            faq_response = extract_faq(mail_content)
            sleep(5)  # API制限対策

            batch_rows.append([
                faq_response["question"],
                faq_response["todo"],
                faq_response["reply"],
            ])

            batch_indexed_files.append([row[THREAD_ID_COLUMN]])  # 1列目をIDとして使用

            # 効率化のためにBatch処理
            if len(batch_rows) >= BATCH_SIZE:
                print("Appending rows to spreadsheet...")
                append_faq(batch_rows)
                append_indexed(batch_indexed_files)
                batch_rows = []
                batch_indexed_files = []

        except Exception as e:
            print(f"Error processing row: {str(e)}")
            continue

    # 残りのバッチを処理
    if batch_rows:
        print("Appending rows to spreadsheet...")
        append_faq(batch_rows)

    if batch_indexed_files:
        append_indexed(batch_indexed_files)

In [ ]:
# 実行
process_mail_faqs()